<a href="https://www.kaggle.com/code/osmanacar/flo-cltv-prediction?scriptVersionId=187666930" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

**FLO-CLTV Prediction**

*  master_id: unique customer number
*  order_channel :the channel where orders are placed
*  last_order_channel : the shopping platform that utilized by the customer (Android, ios, Desktop, Mobile, Offline)
*  first_order_date : customer's first purchase date
*  last_order_date : customer's last purchase date
*  last_order_date_online : customer's last purchase date on online platform
*  last_order_date_offline : customer's last purchase date on offline platform
*  order_num_total_ever_online : total number of purchases by customers on the online platform
*  order_num_total_ever_offline : total number of purchases by customers on the offline platform
*  customer_value_total_ever_offline : total amount of purchases by customers on the offline platform
*  customer_value_total_ever_online : total amount of purchases by customers on the online platform
*  interested_in_categories_12 : Category list for the last 12 months

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
!pip install lifetimes
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.0 MB/s eta 0:00:00


In [2]:
df = pd.read_csv("/kaggle/input/flo-dataset/flo_data_20k.csv")
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


In [3]:
# I define 2 function for outliers
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquartile_range = quartile3 - quartile1
    up_limit = round(quartile3 + 1.5 * interquartile_range)
    low_limit = round(quartile1 - 1.5 * interquartile_range)
    return low_limit, up_limit

def replace_with_threshold(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [6]:
# outliers process for these variables
# order_num_total_ever_online
# order_num_total_ever_offline
# customer_value_total_ever_offline
# customer_value_total_ever_online

columns = ["order_num_total_ever_online", "order_num_total_ever_offline","customer_value_total_ever_offline","customer_value_total_ever_online"]

for col in columns:
    replace_with_threshold(df, col)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.0920,3.8095,1.0000,1.0000,2.0000,4.0000,48.0000
order_num_total_ever_offline,19945.0000,1.8862,1.4347,1.0000,1.0000,1.0000,2.0000,16.0000
customer_value_total_ever_offline,19945.0000,251.9213,251.0237,10.0000,99.9900,179.9800,319.9700,3020.0000
customer_value_total_ever_online,19945.0000,489.7057,632.6098,12.9900,149.9800,286.4600,578.4400,7800.0000


In [8]:
# We will create new variables for each customer's total amount and total number of purchases.
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total"] = df["customer_value_total_ever_online"] + df["customer_value_total_ever_offline"]

In [9]:
# If there is a variable that represents a date, we change the variable type
all_of_date = df.columns[df.columns.str.contains("date")]
df[all_of_date] = df[all_of_date].apply(pd.to_datetime)

df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
order_num_total                             float64
customer_value_total                        float64
dtype: object

**Create the CLTV data structure**

* CLTV = (Customer Value/Churn Rate) X Profit Margin
 
* Customer Value = Average Order Value * Purchase Frequency

* Average Order Value = Total Price / Total Transaction
 
* Purchase Frequency = Total Transaction / Total Number of Customers
 
* Churn Rate = 1 – Repeat Rate

* Profit Margin = Total Price *0.10

In [10]:
# We choose the analysis date 2 days after the last purchase date.
df["last_order_date"].max() # 2021-05-30
today_date = dt.datetime(2021, 6, 1)

In [11]:
# We are creating new dataframe with customer_id, recency_cltv_weekly, T_weekly, frequency and monetary_cltv_average values.
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = (df.apply(lambda x: (x["last_order_date"] - x["first_order_date"]).days, axis=1)) / 7
cltv_df["T_weekly"] = (df.apply(lambda x: (today_date - x["first_order_date"]).days, axis=1)) / 7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_average"] = df["customer_value_total"] / df["order_num_total"]

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_average
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900


In [12]:
# BG/NBD Model
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"], cltv_df["recency_cltv_weekly"], cltv_df["T_weekly"])

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [13]:
# 3 months purchase predict
cltv_df["exp_sales_3_month"] = bgf.predict(4*3, cltv_df["frequency"], cltv_df["recency_cltv_weekly"],cltv_df["T_weekly"])

# 6 months purchase predict
cltv_df["exp_sales_6_month"] = bgf.predict(4*6, cltv_df["frequency"], cltv_df["recency_cltv_weekly"],cltv_df["T_weekly"])

In [14]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_average,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921


In [15]:
# Gamma-Gamma Model
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(cltv_df["frequency"], cltv_df["monetary_cltv_average"])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [16]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"], cltv_df["monetary_cltv_average"])

In [17]:
cltv_df.sort_values(by="exp_average_value", ascending=False).head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_average,exp_sales_3_month,exp_sales_6_month,exp_average_value
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,7.8571,4.0000,1401.8000,1.0944,2.1888,1449.0605
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.7143,101.0000,7.0000,1106.4671,0.7222,1.4445,1127.6115
15516,9083981a-f59e-11e9-841e-000d3a38a36f,63.5714,83.8571,4.0000,1090.3600,0.5746,1.1493,1127.3545
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.4286,2.0000,862.6900,0.7939,1.5878,923.6800
16410,6fecd6c8-261a-11ea-8e1c-000d3a38a36f,57.0000,94.8571,2.0000,859.5800,0.3974,0.7947,920.3579
1853,f02473b0-43c3-11eb-806e-000d3a38a36f,17.2857,23.1429,2.0000,835.8750,0.6843,1.3686,895.0369
7936,ae4ce104-dbd4-11ea-8757-000d3a38a36f,3.7143,42.0000,3.0000,844.3467,0.6766,1.3533,883.2880
9738,3a27b334-dff4-11ea-acaa-000d3a38a36f,40.0000,41.1429,3.0000,837.0567,0.6816,1.3632,875.6741
12828,0c24fc44-2ac8-11ea-9d27-000d3a38a36f,68.0000,84.2857,2.0000,779.2650,0.4235,0.8471,834.5676
2291,26ac1432-1dd3-11ea-8bf2-000d3a38a36f,55.7143,97.7143,3.0000,780.5567,0.4598,0.9197,816.6633


In [18]:
# 6 month CLTV 

cltv = ggf.customer_lifetime_value(
    bgf,
    cltv_df["frequency"],
    cltv_df["recency_cltv_weekly"],
    cltv_df["T_weekly"],
    cltv_df["monetary_cltv_average"],
    time=6, # Month
    freq="W",
    discount_rate=0.01
)

In [19]:
cltv_df["cltv"] = cltv
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_average,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327,395.7332
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650,199.4307
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676,170.2242
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201,98.9455
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251,95.0117


In [20]:
# Create the segments for 6 months CLTV 
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_average,exp_sales_3_month,exp_sales_6_month,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.5714,5.0000,187.8740,0.9739,1.9479,193.6327,395.7332,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,224.8571,21.0000,95.8833,0.9832,1.9663,96.6650,199.4307,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,78.8571,5.0000,117.0640,0.6706,1.3412,120.9676,170.2242,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,20.8571,2.0000,60.9850,0.7004,1.4008,67.3201,98.9455,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.4286,2.0000,104.9900,0.3960,0.7921,114.3251,95.0117,D
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.8571,132.2857,3.0000,66.9533,0.3836,0.7672,71.3481,57.4299,D
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.5714,64.8571,4.0000,93.9825,0.6521,1.3041,98.1334,134.2784,C
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.7143,54.5714,2.0000,81.8150,0.5198,1.0396,89.5702,97.7006,D
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.4286,70.7143,5.0000,210.9380,0.7078,1.4155,217.3009,322.7347,A
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.7143,96.0000,2.0000,82.9800,0.3947,0.7894,90.8146,75.2224,D
